Input: Country Name
Output: Describe the places to go in the Capital city of the country

In [1]:

from langchain_openai import ChatOpenAI
from dotenv import dotenv_values
from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, START, END
from IPython.display import Image, display

config = dotenv_values(".env")

class AgentState(TypedDict):
   country:str
   capital:str
   plan:str
   
llm = ChatOpenAI(api_key=config["OPEN_AI_API_KEY"], model="gpt-4o-mini", temperature=0.1)

def get_capital(agentState: AgentState):
   prompt=f"""What is the captial of {agentState["country"]}.
   just give the city name.
   """
   response=llm.invoke(prompt)
   agentState["capital"]=response.content
   return agentState

def get_plan(agentState: AgentState):
   prompt=f"""What all are the visting places are there in {agentState["capital"]}. give me a plan in 3 stences"""
   response=llm.invoke(prompt)
   agentState["plan"]=response.content
   return agentState

In [2]:
workflow = StateGraph(AgentState)
workflow.add_node("CAPITAL", get_capital)
workflow.add_node("PLAN", get_plan)

workflow.add_edge(START, "CAPITAL")
workflow.add_edge("CAPITAL", "PLAN")
workflow.add_edge("PLAN",END)
graph = workflow.compile()

In [ ]:
display_graph = graph.get_graph()
mermaid_image_data = display_graph.draw_mermaid_png()
display(Image(mermaid_image_data))

In [3]:
agentState = {
   "country" : "India",
   "capital" : "",
   "plan" : ""
}
agentState=graph.invoke(agentState)
print(agentState)

{'country': 'India', 'capital': 'New Delhi.', 'plan': "In New Delhi, you can explore iconic landmarks such as the India Gate, the Red Fort, and the Qutub Minar, along with the serene gardens of Lodhi Garden and the vibrant markets of Chandni Chowk. Start your day with a visit to the historical sites, followed by a leisurely stroll in the gardens, and end with shopping and dining in the bustling markets. Don't forget to take some time to experience the rich culture at the National Museum or the National Gallery of Modern Art."}


Input: Country
Identify The best city to visit
Loop:
    Identify if the weather of the country is cold or hot
    If cold then plan for next best city to visit
Finally:
Give a plan in about 3 lines